In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import tqdm
from PIL import Image
import os
import shutil
import json
import io

# Inspect full image archive without extracting all files

In [4]:
root_path = '../../../validation/image/'
filenames = []
paths = []
for file in os.listdir(root_path)[:]:
    if 'jpg' in file:
        filenames.append(file)
        paths.append(root_path + file)
df = pd.DataFrame({'filename': filenames, 'path': paths})
print(len(df))
df.sample(n=5)

32153


,filename,path
17662,012495.jpg,../../../validation/image/012495.jpg
2460,022405.jpg,../../../validation/image/022405.jpg
26522,001024.jpg,../../../validation/image/001024.jpg
2898,017116.jpg,../../../validation/image/017116.jpg
10389,031586.jpg,../../../validation/image/031586.jpg


# Basic EDA


# Examine classes

In [5]:
%%time

ids = []
sources = []
scales = []
viewpoints = []
zooms = []
occlusions = []
landmarks = []
bboxes = []
styles = []
category_ids = []
category_names = []
annos = [f.replace('image', 'annos').replace('jpg', 'json') for f in df['path']]
print('# rows', len(annos))
for file in tqdm(annos): 
    anno = open(file).read()
    anno = json.loads(anno)
    
    for key in anno.keys():
        if 'item' in key:
            ids.append(file.split('/')[-1].split('.')[0])
            sources.append(anno['source'])
            #anno.keys() => dict_keys(['segmentation', 'scale', 'viewpoint', 'zoom_in', 'landmarks', 'style', 'bounding_box', 'category_id', 'occlusion', 'category_name'])
            scales.append(anno[key]['scale'])
            viewpoints.append(anno[key]['viewpoint'])
            zooms.append(anno[key]['zoom_in'])
            occlusions.append(anno[key]['occlusion'])
            landmarks.append(anno[key]['landmarks'])
            bboxes.append(anno[key]['bounding_box'])
            styles.append(anno[key]['style'])
            category_ids.append(anno[key]['category_id'])
            category_names.append(anno[key]['category_name'])    

# rows 32153


  0%|          | 0/32153 [00:00<?, ?it/s]

CPU times: user 8.4 s, sys: 1.35 s, total: 9.74 s
Wall time: 26.1 s


In [6]:
df_meta_all = pd.DataFrame(data=zip(ids, sources, scales, viewpoints, zooms, occlusions, landmarks, bboxes, styles, category_ids, category_names), columns=['id', 'source', 'scale', 'viewpoint', 'zoom', 'occlusion', 'landmarks', 'bbox', 'style', 'category_id', 'category_name'])
print(len(df_meta_all))
df_meta_all[:5]

52779


,id,source,scale,viewpoint,zoom,occlusion,landmarks,bbox,style,category_id,category_name
0,029122,shop,1,2,1,1,"[447, 482, 1, 398, 482, 2, 421, 530, 2, 453, 5...","[333, 449, 578, 863]",1,10,short sleeve dress
1,025833,shop,2,2,1,3,"[473, 205, 1, 426, 164, 1, 439, 193, 1, 468, 2...","[165, 140, 569, 1024]",1,12,vest dress
2,025833,shop,2,2,1,2,"[476, 201, 1, 426, 159, 1, 442, 185, 2, 473, 2...","[219, 76, 593, 557]",0,1,short sleeve top
3,000995,shop,2,2,1,1,"[313, 189, 1, 283, 190, 2, 300, 201, 2, 316, 2...","[207, 177, 439, 543]",1,10,short sleeve dress
4,024459,shop,1,2,2,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[230, 0, 401, 106]",0,1,short sleeve top


In [7]:
df_meta = df_meta_all.copy()

## Reference:  https://github.com/switchablenorms/DeepFashion2

Key fields:

- source: a string, where 'shop' indicates that the image is from commercial store while 'user' indicates that the image is taken by users.

- scale: a number, where 1 represents small scale, 2 represents modest scale and 3 represents large scale.

- occlusion: a number, where 1 represents slight occlusion(including no occlusion), 2 represents medium occlusion and 3 represents heavy occlusion.

- zoom_in: a number, where 1 represents no zoom-in, 2 represents medium zoom-in and 3 represents lagre zoom-in.

- viewpoint: a number, where 1 represents no wear, 2 represents frontal viewpoint and 3 represents side or back viewpoint.

First pass of training YOLOv5 on DF1 indicates that results including 'user' photos is very poor due to extreme viewing angles (down in mirror at jeans), heavy occlusion, etc.

Therefore, now filtering out 'poor quality' images.

In [8]:
if False:
    df_meta = df_meta[(df_meta.source=='shop') & (df_meta.scale>1) & 
                  (df_meta.occlusion<2) & (df_meta.zoom<2) & 
                  (df_meta.viewpoint<3)]

In [9]:
cnts = df_meta.groupby('category_name').count().sort_values(by='id', ascending=False)
cnts

,id,source,scale,viewpoint,zoom,occlusion,landmarks,bbox,style,category_id
category_name,,,,,,,,,,
short sleeve top,12655,12655,12655,12655,12655,12655,12655,12655,12655,12655
trousers,9759,9759,9759,9759,9759,9759,9759,9759,9759,9759
skirt,6380,6380,6380,6380,6380,6380,6380,6380,6380,6380
long sleeve top,5908,5908,5908,5908,5908,5908,5908,5908,5908,5908
shorts,4322,4322,4322,4322,4322,4322,4322,4322,4322,4322
vest dress,3403,3403,3403,3403,3403,3403,3403,3403,3403,3403
short sleeve dress,3080,3080,3080,3080,3080,3080,3080,3080,3080,3080
long sleeve outwear,2120,2120,2120,2120,2120,2120,2120,2120,2120,2120
vest,2099,2099,2099,2099,2099,2099,2099,2099,2099,2099


In [1]:
# use the same list used for the Training dataset
cats = ['short sleeve top', 'long sleeve top', 'trousers', 'shorts',
       'short sleeve dress', 'long sleeve outwear', 'skirt', 'vest dress',
       'vest', 'long sleeve dress', 'sling dress']

In [11]:
subset = df_meta[df_meta.category_name.isin(cats)]

In [12]:
min_cnt = subset.groupby('category_name').count().min()[0]
min_cnt

1097

In [26]:
# we keep all the data in the validation set for the full dataset
# for the smaller dataset, we target 10% of the size of the training dataset
samples = subset.groupby('category_name').sample(n=500, random_state=314)
len(samples)

5500

In [27]:
df['id'] = df['filename'].apply(lambda x: x.split('.')[0])

In [28]:
#from sklearn.model_selection import train_test_split
#train, val = train_test_split(samples, test_size=0.2)
#print(len(train),len(val))

#For this case, all the data is in the validation set
val = samples

In [29]:
val = pd.merge(df, val, on='id')
len(val)

5500

In [30]:
#train = pd.merge(df, train, on='id')
#train = train[train.folder=='image']
#len(train)

In [31]:
val.iloc[0]

filename                                                029122.jpg
path                          ../../../validation/image/029122.jpg
id                                                          029122
source                                                        shop
scale                                                            1
viewpoint                                                        2
zoom                                                             1
occlusion                                                        1
landmarks        [447, 482, 1, 398, 482, 2, 421, 530, 2, 453, 5...
bbox                                          [333, 449, 578, 863]
style                                                            1
category_id                                                     10
category_name                                   short sleeve dress
Name: 0, dtype: object

In [5]:
root = '../../../'

if not os.path.exists(root+'df2'):
    os.mkdir(root+'df2')

if not os.path.exists(root+'df2/images'):
    os.mkdir(root+'df2/images')
if not os.path.exists(root+'df2/images/train'):
    os.mkdir(root+'df2/images/train')
if not os.path.exists(root+'df2/images/val'):
    os.mkdir(root+'df2/images/val')
if not os.path.exists(root+'df2/images/test'):
    os.mkdir(root+'df2/images/test')

if not os.path.exists(root+'df2/labels'):
    os.mkdir(root+'df2/labels')
if not os.path.exists(root+'df2/labels/train'):
    os.mkdir(root+'df2/labels/train')
if not os.path.exists(root+'df2/labels/val'):
    os.mkdir(root+'df2/labels/val')
if not os.path.exists(root+'df2/labels/test'):
    os.mkdir(root+'df2/labels/test')

In [35]:
target_dim = 640.0
 
def createFiles(data=val, type_='val'):
    for i, row in data.iterrows():
        try:
            filepath = row.path
            img = Image.open(filepath)
            w_orig = float(img.size[0])
            h_orig = float(img.size[1])
            img.thumbnail((target_dim, target_dim)) 
            if 'png' in str(type(img)).lower():
                img = img.convert('RGB')
            img.save(root+'df2/images/'+type_+'/'+row.filename, 'JPEG', optimize=True)

            (x1, y1, x2, y2) = [float(b) for b in row.bbox]
            x = round((((x2 - x1)/2) + x1) / w_orig, 4)
            y = round((((y2 - y1)/2) + y1) / h_orig, 4)
            w = round((x2 - x1) / w_orig, 4)
            h = round((y2 - y1) / h_orig, 4)

            idx = list(cats).index(row.category_name)
            metadata = str(idx) + f' {x} {y} {w} {h}'
            fileroot = root+'df2/labels/'+type_+'/'+row.filename.split('.')[0]+'.txt'

            f = open(fileroot, 'w')
            f.write(metadata)
            f.close()
            img = None
        except:
            print('error:', row.path)

In [36]:
#createFiles(train, type_='train')
createFiles(val, type_='val')

In [6]:
lines = []
lines.append('path: ../df2 # root dir\n')
lines.append('train: images/train\n')
lines.append('val: images/val\n')
lines.append('test:  images/test\n')

lines.append('nc: '+str(len(cats))+' # number of classes\n')

str_classes = "[ '" + "',\n'".join(cats) + "' ]"

lines.append('names: '+str_classes + '\n')

f = open(root+'df2/df2.yaml', 'w')
f.writelines(lines)
f.close()